In [35]:
import os
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GroupKFold
from collections import defaultdict
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import joblib

import matplotlib.pyplot as plt

# SHAP
import shap

# XGBoost (optional)
try:
    import xgboost as xgb
except Exception as e:
    xgb = None
    print("xgboost not available (import failed):", e)

In [36]:
# -------------------------------------------------------------------
# 1. Paths and basic loading
# -------------------------------------------------------------------
TRAIN_PATH = '../data/ML_data/train_panel_years_to_developed.csv'
TEST_PATH  = '../data/ML_data/predict_panel_features.csv'

target_col = 'years_to_developed'
top_k_features = 30   # number of top features to keep (change to 20–30 as you like)

df_train = pd.read_csv(TRAIN_PATH)
df_test  = pd.read_csv(TEST_PATH)

print("Train shape:", df_train.shape)
print("Test shape :", df_test.shape)

# Drop countries from test that also appear in train
if "Country Name" in df_train.columns and "Country Name" in df_test.columns:
    train_countries = set(df_train["Country Name"].unique())
    before = len(df_test)
    df_test = df_test[~df_test["Country Name"].isin(train_countries)].reset_index(drop=True)
    dropped = before - len(df_test)
    print(f"Dropped {dropped} rows from test because Country Name exists in train.")
else:
    print("Warning: 'Country Name' column not found in train or test; no dropping performed.")

# Separate features/target
X_full = df_train.drop(columns=[target_col])
y_full = df_train[target_col]
X_test_full = df_test.copy()


Train shape: (1882, 250)
Test shape : (4358, 249)
Dropped 1758 rows from test because Country Name exists in train.


In [37]:
df_train['Country Name'].unique()

array(['Andorra', 'United Arab Emirates', 'Australia', 'Austria',
       'Belgium', 'Bulgaria', 'Bahrain', 'Bermuda', 'Canada',
       'Switzerland', 'Channel Islands', 'Cyprus', 'Czechia', 'Germany',
       'Denmark', 'Spain', 'Estonia', 'Finland', 'France',
       'Faroe Islands', 'United Kingdom', 'Greece', 'Greenland',
       'Hong Kong SAR, China', 'Croatia', 'Hungary', 'Isle of Man',
       'Ireland', 'Iceland', 'Israel', 'Italy', 'Japan', 'Korea, Rep.',
       'Kuwait', 'Liechtenstein', 'Lithuania', 'Luxembourg', 'Latvia',
       'Monaco', 'Malta', 'Netherlands', 'Norway', 'New Zealand', 'Oman',
       'Poland', 'Portugal', 'Qatar', 'Romania', 'Russian Federation',
       'Saudi Arabia', 'Singapore', 'San Marino', 'Slovak Republic',
       'Slovenia', 'Sweden', 'United States'], dtype=object)

In [38]:
df_test['Country Name'].unique()

array(['Angola', 'Benin', 'Bangladesh', 'Bolivia', 'Bhutan',
       "Cote d'Ivoire", 'Cameroon', 'Congo, Rep.', 'Comoros',
       'Micronesia, Fed. Sts.', 'Ghana', 'Gibraltar', 'Guinea',
       'Honduras', 'Haiti', 'India', 'Kenya', 'Cambodia', 'Kiribati',
       'Lao PDR', 'Sri Lanka', 'Lesotho', 'Myanmar', 'Mauritania',
       'Namibia', 'Nigeria', 'Nicaragua', 'Nepal', 'Philippines',
       'Papua New Guinea', 'Senegal', 'Solomon Islands',
       'Sao Tome and Principe', 'Eswatini', 'Timor-Leste', 'Tanzania',
       'Viet Nam', 'Vanuatu', 'Zambia', 'Zimbabwe'], dtype=object)

In [39]:
# -------------------------------------------------------------------
# 2. Grouped cross-validation by country
#    + SHAP feature selection INSIDE each fold
# -------------------------------------------------------------------

# We keep the original full X/y here
X_full = df_train.drop(columns=[target_col])
y_full = df_train[target_col].values

# Groups = country names
if "Country Name" not in df_train.columns:
    raise ValueError("'Country Name' column is required for grouped CV")

groups = df_train["Country Name"].values

# Identify numeric vs categorical features ON THE FULL DATA
numeric_features_all = X_full.select_dtypes(include=[np.number]).columns.tolist()
categorical_features_all = [c for c in X_full.columns if c not in numeric_features_all]

print("Total numeric features :", len(numeric_features_all))
print("Total categorical features:", len(categorical_features_all))

# GroupKFold: each fold holds out a set of countries
n_splits = 5  # or min(5, number_of_countries) if you want
gkf = GroupKFold(n_splits=n_splits)

# Models as before
models = {
    "linear_regression": LinearRegression(),
    "random_forest": RandomForestRegressor(
        n_estimators=500,
        random_state=42,
        n_jobs=-1
    )
}

if xgb is not None:
    models["xgboost"] = xgb.XGBRegressor(
        n_estimators=500,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        objective="reg:squarederror",
        random_state=42,
        n_jobs=-1
    )

print("\nModels to train:", list(models.keys()))

# To collect metrics and selected features per fold
cv_results = []
fold_selected_features = []

for fold_idx, (train_idx, val_idx) in enumerate(gkf.split(X_full, y_full, groups=groups), start=1):
    print("\n" + "="*60)
    print(f"Fold {fold_idx}/{n_splits}")
    print("="*60)

    # Split by index
    X_tr = X_full.iloc[train_idx].copy()
    y_tr = y_full[train_idx]
    X_val = X_full.iloc[val_idx].copy()
    y_val = y_full[val_idx]

    train_countries = set(df_train.iloc[train_idx]["Country Name"])
    val_countries   = set(df_train.iloc[val_idx]["Country Name"])
    print("Train countries:", len(train_countries))
    print("Val countries  :", len(val_countries))

    # ---------------------------------------------------------------
    # 2a. Feature selection WITHIN THIS FOLD ONLY (SHAP)
    # ---------------------------------------------------------------
    X_tr_num = X_tr[numeric_features_all]

    shap_model = RandomForestRegressor(
        n_estimators=500,
        random_state=42,
        n_jobs=-1
    )
    shap_model.fit(X_tr_num, y_tr)

    sample_size = min(1000, len(X_tr_num))
    X_sample = X_tr_num.sample(n=sample_size, random_state=42)

    explainer = shap.TreeExplainer(shap_model)
    shap_raw = explainer.shap_values(X_sample)

    if isinstance(shap_raw, list):
        shap_matrix = shap_raw[0]
    else:
        shap_matrix = shap_raw

    mean_abs_shap = np.abs(shap_matrix).mean(axis=0)

    shap_importance_fold = (
        pd.DataFrame({
            "feature": numeric_features_all,
            "mean_abs_shap": mean_abs_shap
        })
        .sort_values("mean_abs_shap", ascending=False)
        .reset_index(drop=True)
    )

    top_features_fold = shap_importance_fold["feature"].head(top_k_features).tolist()
    fold_selected_features.append(top_features_fold)

    print(f"Top {top_k_features} SHAP features for fold {fold_idx}:")
    print(top_features_fold)

    # Subset train/val to these top features + any categorical features
    keep_features = top_features_fold + [
        c for c in categorical_features_all if c in X_tr.columns
    ]
    X_tr_fs = X_tr[keep_features]
    X_val_fs = X_val[keep_features]

    # ---------------------------------------------------------------
    # 2b. Preprocessing (defined per fold on selected features)
    # ---------------------------------------------------------------
    numeric_features = X_tr_fs.select_dtypes(include=[np.number]).columns.tolist()
    categorical_features = X_tr_fs.select_dtypes(exclude=[np.number]).columns.tolist()

    numeric_transformer = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ])

    categorical_transformer = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore"))
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, numeric_features),
            ("cat", categorical_transformer, categorical_features)
        ]
    )

    # ---------------------------------------------------------------
    # 2c. Train each model and evaluate on this fold’s validation
    # ---------------------------------------------------------------
    for name, est in models.items():
        print(f"\nTraining {name} on fold {fold_idx}...")

        pipe = Pipeline(steps=[
            ("preprocessor", preprocessor),
            ("model", est)
        ])

        pipe.fit(X_tr_fs, y_tr)
        val_pred = pipe.predict(X_val_fs)

        mse = mean_squared_error(y_val, val_pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_val, val_pred)
        # MAPE is not very meaningful here; keep if you want
        mape = (np.abs((y_val - val_pred) /
                       np.where(y_val == 0, 1e-8, y_val))).mean() * 100
        r2 = r2_score(y_val, val_pred)

        cv_results.append({
            "fold": fold_idx,
            "model": name,
            "rmse": rmse,
            "mae": mae,
            "mape": mape,
            "r2": r2
        })

        print(f"Fold {fold_idx} - {name}: RMSE={rmse:.3f}, MAE={mae:.3f}, R2={r2:.4f}")

# -------------------------------------------------------------------
# 3. Summarize CV metrics across folds
# -------------------------------------------------------------------
cv_results_df = pd.DataFrame(cv_results)
print("\nCross-validated performance (grouped by country):")
display(cv_results_df)

print("\nMean performance by model (across folds):")
display(cv_results_df.groupby("model")[["rmse", "mae", "r2"]].mean())


Total numeric features : 247
Total categorical features: 2

Models to train: ['linear_regression', 'random_forest', 'xgboost']

Fold 1/5
Train countries: 44
Val countries  : 12
Top 30 SHAP features for fold 1:
['Life expectancy at birth, female (years)', 'Fixed telephone subscriptions (per 100 people)', 'Rural population (% of total population)', 'Urban population (% of total population)', 'GDP deflator (base year varies by country)', 'Life expectancy at birth, total (years)', 'Households and NPISHs final consumption expenditure (% of GDP)', 'Population ages 80 and above, male (% of male population)', 'Year.1', 'Year', 'Population ages 35-39, female (% of female population)', 'Arable land (hectares)', 'Population ages 75-79, male (% of male population)', 'Total fisheries production (metric tons)', 'Arable land (% of land area)', 'Mortality rate, neonatal (per 1,000 live births)', 'Merchandise exports (current US$)', 'Agricultural land (% of land area)', 'Official exchange rate (LCU per

,fold,model,rmse,mae,mape,r2
0,1,linear_regression,14.958495,12.020190,3.523033e+09,-0.180846
1,1,random_forest,7.750215,5.380969,1.373523e+09,0.683010
2,1,xgboost,7.854638,5.331133,1.185491e+09,0.674411
3,2,linear_regression,14.421199,11.601854,2.972583e+09,-0.125237
4,2,random_forest,8.236348,6.001920,2.367040e+09,0.632962
5,2,xgboost,7.620719,5.401290,2.037774e+09,0.685780
6,3,linear_regression,11.567515,10.155720,2.874575e+09,0.147407
7,3,random_forest,8.039688,6.427283,1.761123e+09,0.588149
8,3,xgboost,7.544775,5.618891,1.427051e+09,0.637295
9,4,linear_regression,11.431710,10.224691,2.874566e+09,0.159628



Mean performance by model (across folds):


,rmse,mae,r2
model,,,
linear_regression,12.739447,10.848717,0.034191
random_forest,7.416257,5.566721,0.670046
xgboost,7.055905,5.160397,0.702293
